## **1. Unzip Tar File**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tarfile
# Specify the path to your tar file
tar_file_path = "/content/drive/MyDrive/ASAC/00_project/CORP/Dataset/yelp_dataset.tar"
extract_path = "/content/drive/MyDrive/ASAC/00_project/CORP/Dataset"

# Open the tar file
with tarfile.open(tar_file_path) as tar:
    tar.extractall(path=extract_path)
    print(f'successfuly extracted')

## **2. Load Dataset**

In [ ]:
pip install polars

In [ ]:
# 필요한 libarary import
import json
import polars as pl
import pandas as pd
import numpy as np
import requests
import re
from tqdm.auto import tqdm
import time

#viz
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.gridspec as gridspec

#map section
import imageio
import folium
import folium.plugins as plugins

In [ ]:
# Read JSON files
types = ["tip", "checkin", "user", "business"]

file_path = {}
for type in types :
  file_path[type] = f"your_path/yelp_academic_dataset_{type}.json"
  globals()[type] = pl.read_ndjson(file_path[type])
  # by pandas
  # globals()[type] = pd.read_json(file_path[type], lines=True, orient='columns')

In [ ]:
review_path = "your_path/yelp_academic_dataset_review.json"
review = pl.read_ndjson(review_path) # read_json으로 하면 Memory 에러

review = review.sample(n=100000)     # random sampling
review.to_pandas()
review.head()

### Data Check

In [ ]:
## 1) 각 table별 columns 확인
for type in types :
  t = globals()[type]
  print(f"{type}: {t.columns}")

In [ ]:
## 2) 전체 길이, business id & user id에 따른 unique 값 확인
info_list = {}
for type in types :
  t = globals()[type]
  if "business_id" in t and "user_id" in t:
    info_list[type] = [len(t), t["business_id"].n_unique() , t["user_id"].n_unique()]
  elif "user_id" not in t :
    info_list[type] = [len(t), t["business_id"].n_unique(), 0]
  elif "business_id" not in t:
    info_list[type] = [ len(t), 0, t["user_id"].n_unique() ]

info = pd.DataFrame.from_dict(
   info_list, orient = 'index', columns = ["len","business_id","user_id"])
info = info.astype(int)

info["u/b"] = round(info["user_id"]/info["business_id"],2)
info

In [ ]:
## 3) 시각화

# 설정
color = sns.color_palette("Pastel1", len(info.columns))
plt.figure(figsize=(14,8))

# 막대 차트 생성
bar_width = 0.2
for i, idx in enumerate(info.index):
  for j, col in enumerate(info.columns):
    plt.bar(i + j * bar_width, info.loc[idx,col], width = bar_width, color = color[j], align = 'center')
    height = info.loc[idx, col]
    plt.text(i + j * bar_width , height + 0.2, f'{height}', ha = 'center', va = 'bottom',fontsize = 10)

# 레이블, 타이틀 설정
num_bars = len(info.columns)
ticks_positions = [i + (num_bars - 1) * bar_width / 2 for i in range(len(info.index))]
plt.xticks(ticks_positions, info.index, rotation=45, fontsize=12)
plt.yscale('log')
plt.xlabel('Index', fontsize = 16)
plt.ylabel('Count/Ratio', fontsize = 16)

# 범례 설정
legend_label = info.columns
handles = [plt.Rectangle((0,0),1,1,color = color[i]) for i in range(len(legend_label))]
plt.legend(handles, legend_label, title = "Category")

plt.tight_layout()
plt.show()

## **3. EDA**

### Business

레스토랑 카테고리와 그 외 카테고리 분류하여 비교

In [ ]:
# 카테고리 분류
business = business.to_pandas() # pd로 변환
condition = business["categories"].str.upper().str.contains('RESTAURANTS|FOOD|NIGHTLIFE') == True

business_res, business_else = business_pd[condition], business_pd[~condition ]
print(len(business_res), len(business_else))

In [ ]:
## 레스토랑 포함 상위 20개 카테고리 시각화

# 카테고리 결측치 처리
business_res = business_res.fillna("")
categories_res = ','.join(business_res['categories'])
cat_res = pd.DataFrame(categories_res.replace(" ","").split(',') ,columns=['category'])
x_res = cat_res['category'].value_counts()
print(f"There are {len(x)} different types/categories of Businesses in Yelp!")

x_res = x_res.sort_values(ascending=False)
x_res = x_res.iloc[2:22]  # Restaurants, Food 제외

# 시각화
plt.figure(figsize=(16,4))
color = sns.color_palette("summer", len(x))
ax = sns.barplot(x=x_res.index, y=x_res.values, alpha=0.8, palette = color)
plt.title("Top 20 Categories of Restaurants", fontsize=20)
locs, labels = plt.xticks()
plt.setp(labels, rotation=80,fontsize=16)
plt.ylabel('# businesses', fontsize=12)
plt.xlabel('Category', fontsize=12)

# text labels
rects = ax.patches
labels = x_res.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()
# print(list(x_res.index))

In [ ]:
## 그 외 업체 상위 20개 카테고리 시각화

business_else = business_else.fillna("")   # 결측치 채워주기
categories_else = ','.join(business_else['categories'])
cat_else = pd.DataFrame(categories_else.replace(" ","").split(',') ,columns=['category'])
x_els = cat_else['category'].value_counts()
print(f"There are {len(x)} different types/categories of Businesses in Yelp!")

x_els = x_els.sort_values(ascending=False)
x_els = x_els.iloc[0:20]

# 시각화
plt.figure(figsize=(16,4))
color = sns.color_palette("summer", len(x))
ax = sns.barplot(x=x_els.index, y=x_els.values, alpha=0.8, palette = color)
plt.title("Top 20 Categories excecpt Restaurants", fontsize=20)
locs, labels = plt.xticks()
plt.setp(labels, rotation=80)
plt.ylabel('# businesses', fontsize=12)
plt.xlabel('Category', fontsize=12)

# text labels
rects = ax.patches
labels = x_els.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()
# print(list(x_els.index))

In [ ]:
# 평점별 분포 시각화 함수

def ratings(df):
  ratings_df = df.groupby('stars')['review_count'].sum().reset_index()

  x = ratings_df['stars']
  y = ratings_df['review_count']

  plt.figure(figsize=(9,6))
  color = sns.color_palette("BuGn", len(x))
  ax= sns.barplot(x=x, y=y, alpha=0.8, palette = color)
  plt.title(f"Ratings Distribution")
  plt.xlabel('Ratings ', fontsize=12)
  plt.ylabel('Total Review Count', fontsize=12)

  #adding the text labels
  for rect, label in zip(ax.patches, y):
      height = rect.get_height()
      ax.text(rect.get_x() + rect.get_width()/2, height + 5, int(label), ha='center', va='bottom')
  return plt.show()

In [ ]:
ratings(business_res)

map에 좌표 찍어보기

In [ ]:
!pip install folium geopandas

In [ ]:
# 지역별 업체 수
business_rg = business_res.groupby("state").agg(
                  pl.col("city").n_unique().alias("cities"),
                  pl.count("city").alias("businesses"),
                  pl.col("stars").mean().alias("avg_star"),
                  pl.col("review_count").mean().alias("avg_review_cnt"),
                  pl.col("review_count").sum().alias("sum_review_cnt"),
                  pl.col("categories","latitude","longitude"),
                  ).sort("businesses",descending=True)
business_rg.head()  # 27 States 150,346 Businesses

In [ ]:
df = business_rg

# 중심 좌표 설정 -> 모든 위도와 경도의 평균 사용
center_lat = sum([coords[0] for coords in df['latitude']]) / len(df)
center_lon = sum([coords[0] for coords in df['longitude']]) / len(df)

# 지도 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# 데이터프레임의 각 위치를 지도에 추가
for i in range(len(df)):
    folium.Marker(
        location=[df['latitude'][i][0], df['longitude'][i][0]],
        popup=f"{df['city'][i]} (Avg. Stars: {df['avg_star'][i]})",
        tooltip=df['city'][i]
    ).add_to(m)

# 지도 저장
m.save('city_locations_map.html')

# 지도 출력
m

### Checkin

In [ ]:
# data check
idx = 50
print("business_id: ", checkin["business_id"][idx])
point = checkin["date"][idx].split(",")
print("Checkin Counts: ",len(point))
print("Last Checkin: ",point[-1])

In [ ]:
# date 컬럼 리스트화
checkin = checkin.select(
    pl.col("*"),
    pl.col("date").str.extract_all(r"(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})").alias("list"),
)
checkin = checkin.select(
    pl.col("*"), pl.col("list").list.len().alias("ck_cnt")
)
checkin = checkin.sort("count",descending = True) # count 기준으로 정렬
checkin = checkin.to_pandas()  # pd로 변환
checkin

### Join Business & Checkin


In [ ]:
# join
biz_ck = checkin.merge(business_res, on = "business_id", how = "inner")

# review 수와 checkin 수 gap 확인
biz_ck["gap"] = biz_ck["review_count"] - biz_ck["ck_count"]

# checkin 수가 더 많은 경우
biz_ck[biz_ck["gap"] < 0].describe()

In [ ]:
# 그래프 확인
plt.figure(figsize=(10, 6))
plt.plot(biz_ck["ck_count"], biz_ck["review_count"], marker='x', linestyle='-', color='darkcyan')  # 라인 그래프 생성

# 그래프 제목과 축 레이블 설정
plt.title('Line Plot of Date vs Value', fontsize=14)
plt.xlabel('Checkin_Count', fontsize=12)
plt.ylabel('Review_Count', fontsize=12)
plt.grid(True) # 그리드 추가
plt.show()

### 가설검증

가설1 : 업체 수가 많은 주일 수록 평균 평점이 높을 것이다

In [ ]:
business_rg.head(10)  # businesses, avg_star, avg_review_cnt, sum_review_cnt

In [ ]:
# 데이터프레임에서 필요한 열 추출
business_counts = business_rg['businesses'].to_list()
avg_review_counts = business_rg['avg_review_cnt'].to_list()
avg_stars = business_rg['avg_star'].to_list()

# 데이터프레임 생성
data_dict = {
    'businesses': business_counts,
    'avg_review_cnt': avg_review_counts,
    'avg_star': avg_stars
}
df_selected = pd.DataFrame(data_dict)

# Pairplot을 사용하여 상관관계 시각화
sns.pairplot(df_selected)
plt.suptitle('Correlation between Number of Businesses, Average Review Count, and Average Star Rating', y=1.02)
plt.show()

# 상관계수 계산
correlation_matrix = df_selected.corr()
print("Correlation matrix:")
print(correlation_matrix)

가설2 : 전체 checkin 수 or 최근 6개월 내 checkin 수가 많을수록 review 수에 영향을 미칠 것이다
- 2-1
 - checkin은 user가 플랫폼을 통해 직접 활성화 시킨 기록이다.
 - 즉, checkin 수가 많을수록 플랫폼 user 기반 고객이 많이 찾는 가게일 것이다.
 - 플랫폼 user 기반 고객이 많다면 작성된 review 수도 많을 것이다.
- 2-2
 - 최근에 가까운 checkin이 많을수록 상위 노출될 확률이 높아지므로(yelp algorithm),
 - 최근 6개월 내 checkin 많을수록 review 수에 영향을 미칠 것이다.

In [ ]:
## 1) 전체 checkin 기준 상관관계

biz_ck_total = biz_check[['ck_count', 'review_count']].copy()
biz_ck_total.columns = ['checkin_count', 'review_count']

# 상관계수 계산
correlation_matrix = biz_ck_total.corr()
correlation_value = correlation_matrix.loc['checkin_count', 'review_count']
print("Correlation matrix:")
print(correlation_matrix)

# Scatter plot을 사용하여 상관관계 시각화
plt.figure(figsize=(10, 6))
sns.scatterplot(x='checkin_count', y='review_count', data=biz_ck_total)
plt.title('Scatter Plot of Checkin Count vs Review Count', fontsize=14)
plt.xlabel('Checkin Count', fontsize=12)
plt.ylabel('Review Count', fontsize=12)

# 상관계수 값을 그래프에 추가
plt.text(0.05, 0.95, f'Correlation: {correlation_value:.2f}',
         transform=plt.gca().transAxes, fontsize=12, color='red')
plt.grid(True)
plt.show()

In [ ]:
## 2) 최근 6개월 checkin 기준 상관관계

from datetime import datetime, timedelta

# 체크인 날짜 type 변환
all_checkin_dates = [datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for sublist in bis_ck['list'] for date in sublist]

# 기준 날짜를 가장 최근 날짜로 설정
most_recent_date = max(all_checkin_dates)

# 최근 6개월의 기준 날짜 계산
six_months_ago = most_recent_date - timedelta(days=6*30)

# 파생변수 생성 함수
def count_recent_checkins(checkins):
    recent_checkins_count = sum(datetime.strptime(date, '%Y-%m-%d %H:%M:%S') > six_months_ago for date in checkins)
    return recent_checkins_count

# 최근 6개월 이내의 체크인 횟수 계산
biz_ck["recent_checkins_count"] = biz_ck['list'].apply(count_recent_checkins)
biz_ck.sort_values(by="recent_checkins_count", ascending = False)

In [ ]:
# 시각화
biz_ck_recent = df[['review_count', 'recent_checkins_count']].copy()
biz_ck_recent.columns = ['review_count', 'recent_checkins_count']

# 상관계수 계산
correlation_matrix = biz_ck_recent.corr()
correlation_value = correlation_matrix.loc['review_count', 'recent_checkins_count']
print("Correlation matrix:")
print(correlation_matrix)

# Scatter plot을 사용하여 상관관계 시각화
plt.figure(figsize=(10, 6))
sns.scatterplot(x='review_count', y='recent_checkins_count', data=biz_ck_recent)
plt.title('Scatter Plot of Checkin Count vs Review Count', fontsize=14)
plt.xlabel('Review Count', fontsize=12)
plt.ylabel('Recent Checkin', fontsize=12)

# 상관계수 값을 그래프에 추가
plt.text(0.05, 0.95, f'Correlation: {correlation_value:.2f}', transform=plt.gca().transAxes, fontsize=12, color='red')
plt.grid(True)
plt.show()

가설3 : delivery가 가능한 점포의 경우, delivery 리뷰와 그 외 리뷰 구분 가능할 것이다

In [ ]:
# 필요한 컬럼만 추출
review_txt = review[["review_id","user_id","text","stars"]]
review_txt

In [ ]:
# text에 'delivery'가 포함되는 review 따로 확인
condition = review_text["text"].str.lower().str.contains('delivery') == True
review_txt_deli = review_txt[condition]
review_txt_deli

In [ ]:
# good reviews
review_txt_deli[review_txt_deli["stars"]>=4.5]

In [ ]:
# bad reviews
review_txt_deli[review_txt_deli["stars"]<=2]

In [ ]:
def ratings2(df):
  ratings_df = df.groupby('stars').size().reset_index(name="count")

  x = ratings_df['stars']
  y = ratings_df['count']

  plt.figure(figsize=(9,6))
  color = sns.color_palette("BuGn", len(x))
  ax= sns.barplot(x=x, y=y, alpha=0.8, palette = color)
  plt.title(f"Ratings Distribution")
  plt.xlabel('Ratings ', fontsize=12)
  plt.ylabel('Total Review Count', fontsize=12)

  #adding the text labels
  for rect, label in zip(ax.patches, y):
      height = rect.get_height()
      ax.text(rect.get_x() + rect.get_width()/2, height + 5, int(label), ha='center', va='bottom')
  return plt.show()

In [ ]:
ratings2(review_txt_deli)